## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-28-23-45-51 +0000,bbc,Buddhist rites and packed museums: South Korea...,https://www.bbc.com/news/articles/cz0yzxx3x08o...
1,2025-08-28-23-42-21 +0000,startribune,Live: Father of slain child says ‘the hole in ...,https://www.startribune.com/robin-westman-annu...
2,2025-08-28-23-40-30 +0000,bbc,European leaders outraged after Russian strike...,https://www.bbc.com/news/articles/cvg3y7m2gz0o...
3,2025-08-28-23-37-34 +0000,nyt,Thai Court to Rule on Paetongtarn Shinawatra’s...,https://www.nytimes.com/2025/08/28/world/asia/...
4,2025-08-28-23-31-42 +0000,bbc,National security or xenophobia? Texas restric...,https://www.bbc.com/news/articles/cgjyqnndvwjo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
133,trump,49
78,school,17
53,minneapolis,15
60,shooting,13
250,kyiv,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
71,2025-08-28-20-52-21 +0000,nypost,Andrew Cuomo claims ‘President Trump will wind...,https://nypost.com/2025/08/28/us-news/cuomo-cl...,148
47,2025-08-28-21-40-00 +0000,wsj,"Fed governor Lisa Cook sued President Trump, s...",https://www.wsj.com/economy/central-banking/li...,133
284,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...,128
182,2025-08-28-13-46-00 +0000,wsj,"President Trump fired Robert Primus, a board m...",https://www.wsj.com/politics/policy/trump-fire...,102
172,2025-08-28-14-18-31 +0000,cbc,U.S. Fed governor Lisa Cook suing Trump admini...,https://www.cbc.ca/news/world/trump-lisa-cook-...,101


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
71,148,2025-08-28-20-52-21 +0000,nypost,Andrew Cuomo claims ‘President Trump will wind...,https://nypost.com/2025/08/28/us-news/cuomo-cl...
264,74,2025-08-28-03-19-08 +0000,nypost,Russian ballistic missiles hit Kyiv in large-s...,https://nypost.com/2025/08/27/world-news/russi...
50,69,2025-08-28-21-33-28 +0000,nypost,First slain Minneapolis school shooting victim...,https://nypost.com/2025/08/28/us-news/first-sl...
47,68,2025-08-28-21-40-00 +0000,wsj,"Fed governor Lisa Cook sued President Trump, s...",https://www.wsj.com/economy/central-banking/li...
182,42,2025-08-28-13-46-00 +0000,wsj,"President Trump fired Robert Primus, a board m...",https://www.wsj.com/politics/policy/trump-fire...
260,40,2025-08-28-04-20-00 +0000,wsj,"Susan Monarez, the director of the Centers for...",https://www.wsj.com/politics/policy/cdc-direct...
79,34,2025-08-28-20-32-41 +0000,nypost,"UK, EU accuse Putin of ‘sabotaging peace’ afte...",https://nypost.com/2025/08/28/world-news/uk-eu...
23,33,2025-08-28-22-33-31 +0000,startribune,Families identify Fletcher Merkel and Harper M...,https://www.startribune.com/family-of-8-year-o...
243,31,2025-08-28-08-02-39 +0000,nypost,"Vladimir Putin, Kim Jong Un to attend Chinese ...",https://nypost.com/2025/08/28/world-news/putin...
287,31,2025-08-28-00-45-00 +0000,wsj,Tulsi Gabbard surprised CIA officials last wee...,https://www.wsj.com/politics/national-security...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
